## 임베딩 실습
### 활용 데이터: sample.csv

### 실습 개요
이 실습에서는 sample.csv 파일에 포함된 두 개의 텍스트 칼럼에 대해 Sentence-BERT를 활용하여 임베딩 값을 추출하고, 추출된 두 임베딩 간의 코사인 유사도를 계산하는 과제를 수행합니다. Sentence-BERT 라이브러리를 사용하여 텍스트 데이터를 고차원 벡터로 변환하고, 이 벡터들 사이의 유사도를 측정하는 방법을 실습합니다.


### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (pandas, sentence_transformers)를 설치하고 임포트합니다.
2. CSV 파일 로드: pandas를 사용하여 주어진 CSV 파일을 읽고 DataFrame으로 변환합니다.
3. [Sentence-BERT 모델](https://huggingface.co/intfloat/multilingual-e5-large) 로드: sentence_transformers의 SentenceTransformer 클래스를 사용하여 사전 훈련된 모델을 로드합니다.
4. 텍스트 데이터 임베딩: DataFrame 내의 두 텍스트 칼럼에 대해 각각 임베딩을 계산합니다.
5. 코사인 유사도 계산: 추출된 두 임베딩 벡터 간의 코사인 유사도를 계산합니다.
6. 결과 확인 및 저장: 계산된 코사인 유사도를 새로운 열에 저장하고, 최종 결과를 cosine_similarity_results.csv 파일로 저장합니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers sentence-transformers nltk sacrebleu evaluate

In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from scipy.spatial.distance import cosine

In [5]:
df = pd.read_csv('/content/drive/MyDrive/NLP 실습/1 15/sample.csv')

In [6]:
df.head(1)

,sentence1,sentence2
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ..."


In [ ]:
# gpu 설정
model = SentenceTransformer('intfloat/multilingual-e5-large')

In [8]:
# 각 칼럼의 문장들에 대해 임베딩 계산
df['sentence1_embedding'] = df['sentence1'].apply(lambda x: model.encode(x))
df['sentence2_embedding'] = df['sentence2'].apply(lambda x: model.encode(x))

In [10]:
# 코사인 유사도 계산
def calculate_cosine_similarity(row):
    return 1 - cosine(row['sentence1_embedding'], row['sentence2_embedding'])

In [11]:
df['cosine_similarity'] = df.apply(calculate_cosine_similarity, axis=1)

In [12]:
df.head(1)

,sentence1,sentence2,sentence1_embedding,sentence2_embedding,cosine_similarity
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[0.02389471, -0.032740355, -0.029221807, -0.02...","[0.0137563255, -0.024272181, -0.04039307, -0.0...",0.759623


In [ ]:
df.to_csv('cosine_similarity_results.csv', index=False)

## BLEU 및 SacreBLEU 점수 계산 실습

### 활용 데이터: sample.csv

### 실습 개요
이 실습에서는 주어진 csv 파일에 포함된 두 칼럼의 텍스트 데이터를 사용하여 BLEU(Bilingual Evaluation Understudy) 점수와 SacreBLEU 점수를 계산하는 과제를 수행합니다.


### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (pandas, nltk, sacrebleu)를 설치하고 임포트합니다.
2. CSV 파일 로드: pandas를 사용하여 주어진 CSV 파일을 읽고 DataFrame으로 변환합니다.
3. BLEU 점수 계산: Okt 토크나이저를 사용하여 토큰화를 수행한 결과를 기반으로 nltk 라이브러리의 sentence_bleu 함수를 사용하여 BLEU 점수를 계산합니다.
4. SacreBLEU 점수 계산: sacrebleu 라이브러리를 사용하여 SacreBLEU 점수를 계산합니다.
5. 결과 저장 및 비교: 두 방법으로 계산된 BLEU 점수를 각각 새로운 열에 저장하고, 결과를 비교 분석합니다.
6. 최종 결과 저장: 계산된 BLEU 및 SacreBLEU 점수가 포함된 DataFrame을 bleu_sacrebleu_results.csv 파일로 저장합니다.

In [ ]:
!pip install konlpy sacrebleu evaluate sacrebleu[ko]

In [2]:
from nltk.translate.bleu_score import sentence_bleu
from konlpy.tag import Okt
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP 실습/1 15/sample.csv')

In [5]:
df.head(2)

,sentence1,sentence2
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ..."
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어..."


In [6]:
# 토큰화
okt = Okt()

In [7]:
# 각 칼럼의 문장들에 대해 토큰화
df['sentence1_tokenize'] = df['sentence1'].apply(lambda x: okt.morphs(x))
df['sentence2_tokenize'] = df['sentence2'].apply(lambda x: okt.morphs(x))

In [8]:
df = df.reset_index(drop=True)

In [9]:
df['cosine_similarity'] = df.apply(lambda x: len(x['sentence1']) - len(x['sentence2']), axis=1)

In [ ]:
# BLEU 점수 계산 (n-gram 1, 2, 3)
df['bleu_score_1gram'] = df.apply(lambda x: sentence_bleu([x['sentence1_tokenize']], x['sentence2_tokenize'], weights=(1, 0, 0, 0)), axis=1)
df['bleu_score_2gram'] = df.apply(lambda x: sentence_bleu([x['sentence1_tokenize']], x['sentence2_tokenize'], weights=(0.5, 0.5, 0, 0)), axis=1)
df['bleu_score_3gram'] = df.apply(lambda x: sentence_bleu([x['sentence1_tokenize']], x['sentence2_tokenize'], weights=(0.33, 0.33, 0.33, 0)), axis=1)

In [11]:
df.head(2)

,sentence1,sentence2,sentence1_tokenize,sentence2_tokenize,cosine_similarity,bleu_score_1gram,bleu_score_2gram,bleu_score_3gram
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[억제, 돌연변이, 중, NAFM, 733, 은, 0.1, mg, /, ml, 로,...","[응답, 자의, 일상, 활동, 뿐, 만, 아니라, 사회, 경제, 적, ,, 개체군,...",6,0.133333,5.446802e-155,4.575906e-204
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...","[총, 알칼리, 도, ,, 총, 암모늄, ,, tVFA, 와, pH, 의, 농도, ...","[슬라이드, 를, 1,200, rpm, 에서, 5분, 동안, 원심, 분리, 하, 여...",-2,0.159865,8.165163e-02,5.628180e-103


In [12]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-12-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
df['sacrebleu'] = df.apply(lambda x: metric.compute(predictions=[x['sentence1']], references=[[x['sentence2']]], tokenize = "ko-mecab")['score'], axis=1)

In [16]:
df.head(2)

,sentence1,sentence2,sentence1_tokenize,sentence2_tokenize,cosine_similarity,bleu_score_1gram,bleu_score_2gram,bleu_score_3gram,sacrebleu
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[억제, 돌연변이, 중, NAFM, 733, 은, 0.1, mg, /, ml, 로,...","[응답, 자의, 일상, 활동, 뿐, 만, 아니라, 사회, 경제, 적, ,, 개체군,...",6,0.133333,5.446802e-155,4.575906e-204,2.655628
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...","[총, 알칼리, 도, ,, 총, 암모늄, ,, tVFA, 와, pH, 의, 농도, ...","[슬라이드, 를, 1,200, rpm, 에서, 5분, 동안, 원심, 분리, 하, 여...",-2,0.159865,8.165163e-02,5.628180e-103,5.703855
